# Problem2_Regression - Structured_API

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col, sum
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

import matplotlib.pyplot as plt

## Tạo session

In [2]:
spark = SparkSession.builder.appName("BD_Lab03_2.1").getOrCreate()

## Nhập dữ liệu

In [3]:
train_path = "/content/train.csv"
test_path = "/content/test.csv"
train_data = spark.read.csv(train_path, header=True, inferSchema=True)
test_data = spark.read.csv(test_path, header=True, inferSchema=True)

In [4]:
train_data.show(5)
test_data.show(5)

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1| -73.9821548461914| 40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.98041534423828|40.738563537597656|-73.99948120117188| 40.73115158081055|                 N|          663|
|id3858529|        2|2016-01-19 11:35:24|2016-01-19 12:10:48|    

## Xử lý dữ liệu

In [5]:
train_data.select("trip_duration").describe().show()

+-------+------------------+
|summary|     trip_duration|
+-------+------------------+
|  count|            648112|
|   mean| 953.1683227590293|
| stddev|3958.4636896700295|
|    min|                 1|
|    max|           1939736|
+-------+------------------+



Có thể thấy được rằng độ lệch chuẩn của trip_duration rất cao => xuất hiện outlier. Cần phải lọc lại dữ liệu.

## Loại bỏ outliers

In [6]:
# Tính Q1 và Q3
quantiles = train_data.approxQuantile("trip_duration", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Lọc bỏ outliers
clean_data = train_data.filter((col("trip_duration") >= lower_bound) & (col("trip_duration") <= upper_bound))

In [7]:
train_data = clean_data
train_data.select("trip_duration").describe().show()

+-------+----------------+
|summary|   trip_duration|
+-------+----------------+
|  count|          614375|
|   mean|730.054691353001|
| stddev|445.827222197628|
|    min|               1|
|    max|            2072|
+-------+----------------+



Sau khi loại bỏ outlier, thì độ lệch chuẩn đã giảm đi rất nhiều.

## Nhóm các cột numeric thành 1 vector

In [8]:
assembler = VectorAssembler(
    inputCols=["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"],
    outputCol="features"
)
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

In [9]:
train_data = train_data.select("features", "trip_duration")

## Chia tập dữ liệu để validation

In [10]:
train_train_data, val_train_data = train_data.randomSplit([0.8, 0.2], seed=42)

## Sử dụng mô hình DecisionTreeRegressor

In [11]:
dt = DecisionTreeRegressor(featuresCol="features",
                            labelCol="trip_duration",
                              maxDepth=20,
                                impurity="variance")
model = dt.fit(train_train_data)


In [17]:
importances = model.featureImportances
cols_name = ["passenger_count", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]

print("Feature Importances:")
importances_list = importances.toArray()
for i in range(len(cols_name)):
  print(f"{cols_name[i]}: {importances_list[i]:6f}")


Feature Importances:
passenger_count: 0.017552
pickup_longitude: 0.214884
pickup_latitude: 0.330539
dropoff_longitude: 0.198280
dropoff_latitude: 0.238744


| Đặc trưng             | Độ quan trọng | Nhận xét                                                                 |
|------------------------|----------------|-------------------------------------------------------------------------|
| `pickup_latitude`      | **0.3305**     | Là đặc trưng quan trọng nhất.    |
| `dropoff_latitude`     | 0.2387         | Cũng rất quan trọng.             |
| `pickup_longitude`     | 0.2149         | Có vai trò lớn.                                |
| `dropoff_longitude`    | 0.1983         | Gần tương đương pickup_longitude.                     |
| `passenger_count`      | **0.0176**     | Ảnh hưởng rất nhỏ, gần như không giúp ích nhiều cho mô hình.          |



## Đánh giá mô hình

In [13]:
predictions = model.transform(val_train_data)

In [14]:
evaluator_rmse = RegressionEvaluator(labelCol="trip_duration", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="trip_duration", predictionCol="prediction", metricName="r2")

rmse = evaluator_rmse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"RMSE: {rmse:6f}")
print(f"R²: {r2:6f}")


RMSE: 294.281568
R²: 0.565827


- RMSE ở mức khá cao, có thể do dữ liệu có độ phân tán lớn hoặc còn thiếu đặc trưng quan trọng.
- R² chưa đạt mức cao, mô hình chưa thể hiện tốt mối quan hệ giữa các đặc trưng.

In [18]:
predictions = model.transform(test_data)

In [26]:
output = predictions.select(col("id"), col("prediction"))

In [27]:
output.toPandas().to_csv("results.csv", index=False)